# Hugging Face를 이용한 모델 생성 및 평가 예제

<br><br>

#### 초록
***
<span style="font-size:11pt; line-height:1.8;">
    &nbsp; &nbsp; Hugging Face의 transformers를 이용하여 BERT, RoBERTa 모델과 토크나이저를 불러와 MNLI 데이터를 학습하고 각 모델의 성능을 확인하였다. 학습 데이터는 총 392,702개이며, 검증 데이터와 학습 데이터는 각 항목 별로 약 9,700개 이다. 검증 데이터와 학습 데이터는, 학습 데이터에 포함된 도메인 자료인 Matched 데이터와 포함 되지 않은 Mismatched 데이터로 구분 된다. 데이터 전처리는 '검증 및 테스트 데이터 분할',  '모델별 토크나이저를 이용한 학습 데이터 생성' 순으로 진행 하였다. BERT와 RoBERTa 모델을 생성하고 Adam을 이용하여 총 5회 학습 하였다. 학습한 모델을 테스트 데이터로 평가한 결과, BERT 모델의 Matched 테스트 데이터의 정확도는 0.435 이고 Mismatched 테스트 데이터의 정확도는 0.460 이다. RoBERTa 모델의 Matched 테스트 데이터의 정확도는 0.534 이고 Mismatched 테스트 데이터의 정확도는 0.537 이다.
</span>

***

<br>

## 1. 서론
***
<span style="font-size:12pt; line-height:1.7; font-family:Serif;">
    &nbsp; &nbsp; 본 예제는 NLP 프레임워크 중 하나인 Hugging Face의 transformers를 이용하여 Multi-Genre Natural Language Inference(MNLI) 문제를
해결하고자 합니다. Hugging Face의 transformers의 경우 학습된 모델과 토크나이저를 사용할 수 있다는 이점이 있습니다. 또한 사전학습된 transformers의 BERT와 RoBERTa 모델과 토크나이저를 이용하여 두 모델의 성능을 비교하고자 합니다. 다음은 예제의 진행 과정을 제시한 것입니다.
</span><br><br>


> _1. 데이터 분석_
>
> _2. 데이터 전처리_
>
> _3. 모델 생성_
>
> _4. 모델 학습 및 평가_


***

#### _들어가기 앞서..._
***
+ `tensorflow_datasets`의 `glue/mnli` 데이터셋을 받기 위해 라이브러리 버전을 업그레이드 합니다.

`$pip install tensorflow-datasets -U`
***

<br>

## 2. 데이터 분석
***
<span style="font-size:12pt; line-height:1.7; font-family:Serif;">
    &nbsp; &nbsp; GLUE의 MNLI는 전제 문장(Premise)과 가설 문장(Hypothesis)이 주어졌을 때, 전제와 가설의 관계를 구분(class)하는 것을 목표로 합니다. 전제와 가설의 관계는 수반(entailment), 모순(contradiction), 중립(neutral)으로 총 세 개로 구분 됩니다. 데이터는 연설, 소설, 기관 보고서 등 10개 도메인의 자료를 사용 합니다. 데이터는 학습, 검증, 테스트 데이터가 있으며, 검증 및 테스트 데이터의 경우, 학습 데이터에 포함되지 않은(mismatched) 도메인의 자료에 해당하는 데이터와 학습 데이터에 포함된(matched) 도메인의 자료로 구분되어 있습니다. 학습 데이터는 총 392,702개이며, 검증 데이터와 학습 데이터는 각 항목 별로 약 9,700개 입니다.
</span>

***

#### 필요 라이브러리 호출
***
+ 예제에서 사용할 라이브러리를 호출 합니다.
***

In [1]:
import os   #디렉토리 관리


import tensorflow as tf   #신경망
import tensorflow_datasets as tfds   #데이터셋
from dataclasses import asdict   #데이터 클래스


from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures   #허깅 페이스
from transformers import BertTokenizer, TFBertForSequenceClassification   #BERT
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification   #RoBerta

<br>

#### MNLI 데이터 불러오기
***
+ 텐서플로우 데이터셋을 이용하여 GLUE MNLI 데이터셋을 불러옵니다.


+ 학습 데이터는 392,702개 입니다.


+ 학습 데이터에 포함된(matched) 도메인의 검증 데이터는 9,815개, 테스트 데이터는 9,796개 입니다.


+ 학습 데이터에 포함 되지 않은(mismatched) 도메인의 검증 데이터는 9,832개, 테스트 데이터는 9,847개 입니다.
***

In [2]:
dataset, info = tfds.load('glue/mnli', with_info=True)

print("=" * 100)
for idx, (domain, data) in enumerate(dataset.items()):
    print(f"{idx}: {domain}({len(data):,})")
print("=" * 100)

INFO:absl:Load dataset info from /aiffel/tensorflow_datasets/glue/mnli/2.0.0
INFO:absl:Reusing dataset glue (/aiffel/tensorflow_datasets/glue/mnli/2.0.0)
INFO:absl:Constructing tf.data.Dataset glue for split None, from /aiffel/tensorflow_datasets/glue/mnli/2.0.0


0: train(392,702)
1: validation_matched(9,815)
2: validation_mismatched(9,832)
3: test_matched(9,796)
4: test_mismatched(9,847)


#### 학습 데이터 출처
***
+ TensorFlow, GLUE MNLI(2019), https://www.tensorflow.org/datasets/catalog/glue
***

<br>

#### MNLI 데이터 샘플 출력
***
+ MNLI 데이터 중 레코드 하나를 출력 합니다.


+ MNLI 데이터는 전제 문장(Premise)와 가설 문장(Hypothesis) 그리고 두 문장의 관계에 대한 값(Label)로 이루어져 있습니다.


+ Label은 모순(contradiction: 0), 수반(entailment: 1), 중립(neutral: 2)으로 총 세 개 입니다.
***

In [3]:
print("=" * 100)
for data in dataset['train']:
    print("Hypothesis:", data['hypothesis'].numpy(), end="\n\n")
    print("Premise:", data['premise'].numpy(), end="\n\n")
    print("Label:", data['label'].numpy())
    break
print("=" * 100)

Hypothesis: b'Meaningful partnerships with stakeholders is crucial.'

Premise: b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'

Label: 1


<br>

## 3. 데이터 전처리
***
<span style="font-size:12pt; line-height:1.7; font-family:Serif;">
    &nbsp; &nbsp; 데이터 전처리는 '검증 및 테스트 데이터 분할', 'MNLI Processor 생성', '토크나이저를 이용한 학습 데이터 생성' 순으로 진행 됩니다. 검증 데이터의 경우, Matched 데이터와 Mismatched 데이터를 합하여 이를 학습 시, 검증에 사용 합니다. MNLI Processor를 이용하여 데이터를 모델에 입력하기 적합하도록 전처리 하고 BERT와 RoBERTa 토크나이저를 이용하여 각 모델에 입력하기 위한 데이터셋을 생성 합니다.
</span>

***

#### 검증 및 테스트 데이터 섞기 및 분할
***
+ 검증(validation) 데이터와 테스트(test) 데이터를 섞어준 후, 다시 분할 하여 줍니다.


+ 검증 데이터의 경우 matched 데이터와 mismatched 데이터를 결합하여 학습 시, 모델의 성능을 확인 합니다.
***

In [4]:
def split_dataset(dataset, val_size):    
    dataset = dataset.shuffle(len(dataset))
    val_dataset = dataset.shuffle(len(dataset)).take(val_size)
    test_dataset = dataset.shuffle(len(dataset)).skip(val_size)
    return (val_dataset, test_dataset)


matched_data = dataset['test_matched'].concatenate(dataset['validation_matched'])
mismatched_data = dataset['test_mismatched'].concatenate(dataset['validation_mismatched'])

matched_val, matched_test = split_dataset(matched_data, len(dataset['validation_matched']))
mismatched_val, mismatched_test = split_dataset(mismatched_data, len(dataset['validation_mismatched']))

val = mismatched_val.concatenate(matched_val)


print("=" * 100)
print(f"matched_val Num: {len(matched_val):,}\tmismatched_val Num: {len(mismatched_val):,}")
print(f"matched_test Num: {len(matched_test):,}\tmismatched_test Num: {len(mismatched_test):,}")
print("=" * 100)

matched_val Num: 9,815	mismatched_val Num: 9,832
matched_test Num: 9,796	mismatched_test Num: 9,847


<br>

#### MNLI Processor 생성
***
+ dataset을 모델에 입력하기에 적합하도록 전처리 하는 역할의 MNLI Processor를 생성 합니다.


+ 추상 클래스인 DataProcessor를 생성한 후, 이를 MNLI Processor에 상속 합니다.
***

In [5]:
#추상 클래스=============================================
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))
#End=====================================================


#MNLI Processor==========================================
class MnliProcessor(DataProcessor):
    """Processor for the MRPC data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev_matched.tsv")), "dev_matched")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test_matched.tsv")), "test_matched")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples
#End=====================================================

<br>

#### MNLI Processor 예제 출력
***
+ 기존 dataset데이터를 MNLI Processor를 이용하여 전처리한 데이터를 출력 합니다.


+ 기존 Dictionary 형태의 데이터가 transformers의 Processor 형태로 변환된 것을 확인할 수 있습니다.


+ Label의 경우, 세 개의 클래스가 올바르게 적용된 것을 확인할 수 있습니다.
***

In [6]:
processor = MnliProcessor()
examples = dataset['train'].take(1)

for example in examples:
    print("Original Data".center(100, "="))
    print(example, end="\n\n")
    print(type(example))
    print("=" * 100, end="\n\n")
    
    print("Processed Data".center(100, "="))
    example = processor.get_example_from_tensor_dict(example)
    print(example, end="\n\n")
    print(type(example))
    print("=" * 100, end="\n\n")
    
    label_map = {label: i for i, label in enumerate(processor.get_labels())}
    print("Label", label_map)

===========================================Original Data============================================
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>}

<class 'dict'>

===========================================Processed Data===========================================
InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnersh

<br>

#### 문장 정수 인코딩 및 데이터셋 생성 함수 정의
***
+ 토크나이저를 이용하여 데이터를 정수화 하는 함수와 정수화한 데이터를 Tensorflow 데이터셋으로 생성하는 함수를 정의 합니다.
***

In [7]:
#정수 인코딩==========================================
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    return features
#End==================================================


#tf_dataset===========================================
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )
#End==================================================

<br>

#### 토크나이저 불러오기
***
+ BERT와 RoBERTa의 MNLI에 대한 성능을 확인 하기 위해, Transformers로부터 BERT와 RoBERTa의 토크나이저를 불러옵니다.
***

In [8]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base", use_fast=True)

<br>

#### 학습 데이터셋 생성
***
+ 앞서 정의한 전처리 함수와 토크나이저를 이용하여 BERT와 RoBerta를 학습하기 위한 데이터셋을 생성 합니다.

***

In [9]:
def get_dataset_batch(raw_dataset_list, tokenizer, processor):
    dataset_list = []
    for idx, dataset in enumerate(raw_dataset_list):
        data = tf_glue_convert_examples_to_features(dataset, tokenizer, max_length=128, processor=processor)
        
        if idx == 0:
            data_batch = data.shuffle(100).batch(16).repeat(2)
        else:
            data_batch = data.shuffle(100).batch(16)
        dataset_list.append(data_batch)
    return dataset_list
    
    
raw_dataset_list = (dataset['train'], val, matched_test, mismatched_test)

bert_train, bert_val, bert_m_test, bert_mism_test = get_dataset_batch(
    raw_dataset_list, bert_tokenizer, processor
)

robert_train, robert_val, robert_m_test, robert_mism_test = get_dataset_batch(
    raw_dataset_list, roberta_tokenizer, processor
)

<br>

## 4. 모델 생성 및 학습
***
<span style="font-size:12pt; line-height:1.7; font-family:Serif;">
    &nbsp; &nbsp; BERT와 RoBERTa 모델을 생성하고 MNLI 데이터셋을 학습 합니다. RoBERTa는 Robustly Optimized BERT Pretraining Approach로 BERT의 pre-training 단계의 하이퍼 파라미터를 다양하게 변경하여 최적화한 모델에 해당 합니다. RoBERTa의 특징은 pre-train 시에 160GB의 학습 데이터를 이용하였으며, GLUE, SQuAD, RACE 문제를 중심으로 학습 하였습니다. 또한, 배치 사이즈가 클수록 성능이 좋아져, 기존 BERT의 3배 크기의 배치 사이즈로 학습 하였습니다. 학습 옵티마이저는 Adam을 이용하고 학습 회수는 5회로 설정 합니다.
</span>

***

#### 옵티마이저 설정
***
+ 옵티마이저의 경우 BERT와 RoBERTa에 동일하게 적용되기 때문에 미리 정의하여 줍니다.


+ 옵티마이저는 Adam을 이용합니다.
***

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

<br>

### 4.1. BERT
***
<span style="font-size:12pt; line-height:1.7; font-family:Serif;">
    &nbsp; &nbsp; BERT 모델을 생성하고 MNLI 데이터셋을 학습 합니다. 모델의 파라미터 크기는 109,484,547 입니다. 총 5회 학습한 결과, 최종 검증 손실값은 1.080이고 정확도는 0.514 입니다.
</span>

***

#### BERT 모델 생성
***
+ 모델을 생성하고 앞서 정의한 옵티마이저를 적용합니다.


+ 모델의 파라미터 크기는 109,484,547 입니다.
***

In [11]:
bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
bert_model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
bert_model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


<br>

#### BERT 모델 학습
***
+ 앞서 생성한 데이터셋을 바탕으로 BERT 모델을 학습 합니다.


+ 학습 회수는 총 5회로 설정 합니다.
***

In [12]:
bert_model.fit(bert_train, epochs=5, steps_per_epoch=115, validation_data=bert_val)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


115/115 [==============================] - ETA: 0s - loss: 1.0892 - acc: 0.3814

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


115/115 [==============================] - 442s 4s/step - loss: 1.0888 - acc: 0.3821 - val_loss: 1.1026 - val_acc: 0.4277
Epoch 2/5
115/115 [==============================] - 422s 4s/step - loss: 0.8935 - acc: 0.5872 - val_loss: 1.0863 - val_acc: 0.4778
Epoch 3/5
115/115 [==============================] - 422s 4s/step - loss: 0.8019 - acc: 0.6598 - val_loss: 1.0590 - val_acc: 0.4817
Epoch 4/5
115/115 [==============================] - 422s 4s/step - loss: 0.7459 - acc: 0.6850 - val_loss: 1.1507 - val_acc: 0.5029
Epoch 5/5
115/115 [==============================] - 422s 4s/step - loss: 0.7582 - acc: 0.6634 - val_loss: 1.0802 - val_acc: 0.5144


<br>

### 4.2. RoBERTa
***
<span style="font-size:12pt; line-height:1.7; font-family:Serif;">
    &nbsp; &nbsp; RoBERTa 모델을 생성하고 MNLI 데이터셋을 학습 합니다. 모델의 파라미터 크기는 124,647,939 입니다. 총 5회 학습한 결과, 최종 검증 손실값은 1.358이고 정확도는 0.536 입니다.
</span>

***

#### RoBERTa 모델 생성
***
+ 모델을 생성하고 앞서 정의한 옵티마이저를 적용합니다.


+ 모델의 파라미터 크기는 124,647,939 입니다.
***

In [11]:
roberta_model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
roberta_model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
roberta_model.summary()

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124055040 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  592899    
Total params: 124,647,939
Trainable params: 124,647,939
Non-trainable params: 0
_________________________________________________________________


<br>

#### RoBERTa 모델 학습
***
+ 앞서 생성한 데이터셋을 바탕으로 RoBERTa 모델을 학습 합니다.


+ 학습 회수는 총 5회로 설정 합니다.
***

In [12]:
roberta_model.fit(robert_train, epochs=5, steps_per_epoch=115, validation_data=robert_val)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


115/115 [==============================] - ETA: 0s - loss: 1.0922 - acc: 0.3681

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


115/115 [==============================] - 438s 4s/step - loss: 1.0922 - acc: 0.3682 - val_loss: 1.1054 - val_acc: 0.1580
Epoch 2/5
115/115 [==============================] - 421s 4s/step - loss: 1.1014 - acc: 0.3455 - val_loss: 1.0801 - val_acc: 0.3372
Epoch 3/5
115/115 [==============================] - 419s 4s/step - loss: 0.9660 - acc: 0.5556 - val_loss: 1.0942 - val_acc: 0.4741
Epoch 4/5
115/115 [==============================] - 419s 4s/step - loss: 0.7794 - acc: 0.6777 - val_loss: 1.0392 - val_acc: 0.5808
Epoch 5/5
115/115 [==============================] - 419s 4s/step - loss: 0.7092 - acc: 0.7068 - val_loss: 1.3580 - val_acc: 0.5357


<br>

## 5. 모델 평가
***
<span style="font-size:12pt; line-height:1.7; font-family:Serif;">
    &nbsp; &nbsp; 학습된 각 모델을 테스트 데이터를 이용하여 평가 합니다. 테스트 데이터의 경우 학습 데이터에 포함되지 않은 도메인 자료인 Mismatched 데이터 9,847개와 포함된 도메인 자료 데이터인 Matched 데이터 9,796개로 이루어져 있습니다. 학습 데이터에 포함되지 않은 자료를 바탕으로 모델을 학습하는 것은 모델이 학습하지 않은 도메인에 대한 평가가 이루어지기 때문에 모델의 일반화 능력을 확인할 수 있습니다.
</span><br><br>

<span style="font-size:12pt; line-height:1.7; font-family:Serif;">
    &nbsp; &nbsp; 테스트 데이터에 대한 BERT 모델 성능을 확인한 결과 Matched 테스트 데이터의 손실값은 1.032, 정확도는 0.435이고, Mismatched 테스트 데이터의의 손실값은 1.011, 정확도는 0.460 입니다. 테스트 데이터에 대한 RoBERTa 모델 성능을 확인한 결과 Matched 테스트 데이터의 손실값은 1.352, 정확도는 0.534이고, Mismatched 테스트 데이터의의 손실값은 1.355, 정확도는 0.537 입니다.
</span>

***

#### 테스트 데이터에 대한 BERT 모델 성능 확인
***
+ 학습 데이터에 포함된 도메인의 테스트 데이터(Matched)와 포함되지 않은 테스트 데이터(MisMatched)를 이용하여 모델의 성능을 확인 합니다.


+ Matched 테스트 데이터의 손실값은 1.032이며, Mismatched 테스트 데이터의 손실값은 1.011 입니다.


+ Matched 테스트 데이터의 정확도은 0.435이며, Mismatched 테스트 데이터의 정확도은 0.460 입니다.
***

In [33]:
matched_result = bert_model.evaluate(bert_m_test)
mismatched_result = bert_model.evaluate(bert_mism_test)

print(f"\nMatched Test data\tLoss: {matched_result[0]:.3f}\tAccuracy: {matched_result[1]:.3f}")
print(f"Mismatched Test data\tLoss: {mismatched_result[0]:.3f}\tAccuracy: {mismatched_result[1]:.3f}")

616/616 [==============================] - 167s 271ms/step - loss: 1.0106 - acc: 0.4600

Matched Test data	Loss: 1.032	Accuracy: 0.435
Mismatched Test data	Loss: 1.011	Accuracy: 0.460


<br>

#### 테스트 데이터에 대한 RoBERTa 모델 성능 확인
***
+ 학습 데이터에 포함된 도메인의 테스트 데이터(Matched)와 포함되지 않은 테스트 데이터(MisMatched)를 이용하여 모델의 성능을 확인 합니다.


+ Matched 테스트 데이터의 손실값은 1.352이며, Mismatched 테스트 데이터의 손실값은 1.355 입니다.


+ Matched 테스트 데이터의 정확도는 0.534이며, Mismatched 테스트 데이터의 정확도는 0.537 입니다.
***

In [13]:
matched_result = roberta_model.evaluate(robert_m_test)
mismatched_result = roberta_model.evaluate(robert_mism_test)

print(f"\nMatched Test data\tLoss: {matched_result[0]:.3f}\tAccuracy: {matched_result[1]:.3f}")
print(f"Mismatched Test data\tLoss: {mismatched_result[0]:.3f}\tAccuracy: {mismatched_result[1]:.3f}")

616/616 [==============================] - 166s 269ms/step - loss: 1.3546 - acc: 0.5366

Matched Test data	Loss: 1.352	Accuracy: 0.534
Mismatched Test data	Loss: 1.355	Accuracy: 0.537


<br>

## 6. 결론
***
<span style="font-size:12pt; line-height:1.7; font-family:Serif;">
    &nbsp; &nbsp; Hugging Face의 transformers를 이용하여 BERT, RoBERTa 모델과 토크나이저를 불러와 MNLI 데이터를 학습하고 각 모델의 성능을 확인하였습니다. Hugging Face는 NLP 프레임워크로, pre-trained 모델과 토크나이저를 쉽게 사용할 수 있습니다. MNLI 데이터를 transformers Processor를 이용하여 전처리 한 후, 각 모델을 학습하였습니다. 두 모델 모두 Adam 옵티마이저를 사용하였으며, 총 5회 학습 하였습니다. 학습된 모델을 테스트 데이터로 평가한 결과, BERT 모델의 Matched 테스트 데이터의 정확도는 0.435 이고 Mismatched 테스트 데이터의 정확도는 0.460 입니다. RoBERTa 모델의 Matched 테스트 데이터의 정확도는 0.534 이고 Mismatched 테스트 데이터의 정확도는 0.537 입니다. 따라서 RoBERTa가 BERT 보다 성능이 우세하다 고 할 수 있습니다. [표 1]은 BERT와 RoBERTa 모델의 성능 지표를 제시한 것입니다.
</span><br><br>


|Model|Validation Loss|Validation Accuracy|Matched Test Loss|Matched Test Accuracy|Mismatched Test Loss|Mismatched Test Accuracy|
|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
|**BERT**|1.080|0.514|1.032|0.435|1.011|0.460|
|**RoBERTa**|1.358|**0.536**|1.352|**0.534**|1.355|**0.537**|

[표 1] BERT, RoBERTa 모델의 성능 지표

***

#### 형상관리 기록
***
+ v1_1: 실습 예제 진행


+ v1_2: 제출 예제 진행


+ v2_1: BERT 모델 사용


+ v3_1: RoBerta 모델 사용
***